In [27]:
# !pip install langchain
# !pip install openai
# !pip install pypdf
# !pip install chromadb
# !pip install langchainhub

In [28]:
from langchain.document_loaders.csv_loader import CSVLoader

In [29]:
loader = CSVLoader(file_path="./assets-resources/superheroes.csv")
data = loader.load()
data

[Document(page_content='Superhero Name: Captain Thunder\nSuperpower: Bolt Manipulation\nPower Level: 90\nCatchphrase: Feel the power of the storm!', metadata={'source': './assets-resources/superheroes.csv', 'row': 0}),
 Document(page_content='Superhero Name: Silver Falcon\nSuperpower: Flight and Agility\nPower Level: 85\nCatchphrase: Soar high, fearlessly!', metadata={'source': './assets-resources/superheroes.csv', 'row': 1}),
 Document(page_content='Superhero Name: Mystic Shadow\nSuperpower: Invisibility and Illusions\nPower Level: 78\nCatchphrase: Disappear into the darkness!', metadata={'source': './assets-resources/superheroes.csv', 'row': 2}),
 Document(page_content='Superhero Name: Blaze Runner\nSuperpower: Pyrokinesis\nPower Level: 88\nCatchphrase: Burn bright and fierce!', metadata={'source': './assets-resources/superheroes.csv', 'row': 3}),
 Document(page_content='Superhero Name: Electra-Wave\nSuperpower: Electric Manipulation\nPower Level: 82\nCatchphrase: Unleash the electri

In [30]:
import pandas as pd

file_path="./assets-resources/superheroes.csv"
df = pd.read_csv(file_path)

df.head()

,Superhero Name,Superpower,Power Level,Catchphrase
0,Captain Thunder,Bolt Manipulation,90,Feel the power of the storm!
1,Silver Falcon,Flight and Agility,85,"Soar high, fearlessly!"
2,Mystic Shadow,Invisibility and Illusions,78,Disappear into the darkness!
3,Blaze Runner,Pyrokinesis,88,Burn bright and fierce!
4,Electra-Wave,Electric Manipulation,82,Unleash the electric waves!


In [31]:

from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    model="mistral",
    verbose=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

In [33]:
# Embed and store
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [34]:
question = "What is the name of the thunder super hero?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [35]:
docs

[Document(page_content='Superhero Name: Thunderstrike\nSuperpower: Lightning Control\nPower Level: 91\nCatchphrase: Electrify the battlefield!', metadata={'row': 30, 'source': './assets-resources/superheroes.csv'}),
 Document(page_content='Superhero Name: Thunderstrike\nSuperpower: Lightning Control\nPower Level: 91\nCatchphrase: Electrify the battlefield!', metadata={'row': 30, 'source': './assets-resources/superheroes.csv'}),
 Document(page_content='Superhero Name: Thunderstrike\nSuperpower: Lightning Control\nPower Level: 91\nCatchphrase: Electrify the battlefield!', metadata={'row': 30, 'source': './assets-resources/superheroes.csv'}),
 Document(page_content='Superhero Name: Captain Thunder\nSuperpower: Bolt Manipulation\nPower Level: 90\nCatchphrase: Feel the power of the storm!', metadata={'row': 0, 'source': './assets-resources/superheroes.csv'})]

In [36]:
# QA chain
from langchain.chains import RetrievalQA

# RAG prompt
from langchain import hub

QA_CHAIN_PROMPT = hub.pull("rlm/rag-prompt-llama")

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

question = "What is the catch phrase for the super hero with the power of thunder?"
result = qa_chain({"query": question})


The catchphrase for Captain Thunder, who has the power of bolt manipulation and a power level of 90, is "Feel the power of the storm!".